In [10]:
import pandas as pd
import numpy as np
import os
import re

In [11]:
def get_taxa_columns(df,rank):
    '''Splits ID into taxanomic ranks to make taxa table'''
    df2 = df.copy()
    df_taxa = df2['clade_name'].str.split('|',expand=True)
    taxa_cols = ["Kingdom","Phylum","Class","Order","Family","Genus","Species","Strain"]
    taxa_dict = {'Kingdom':1,"Phylum":2,"Class":3,"Order":4,"Family":5,"Genus":6,"Species":7,"Strain":8}
    value = taxa_dict.get(rank)
    taxa_cols=taxa_cols[0:value]
    df_taxa.columns=taxa_cols
    for col in df_taxa.columns:
        df_taxa[col]=df_taxa[col].apply(trim_taxa_names)    
    otu_index = []
    for i in range(0, len(df)):
        otu_index.append("OTU"+str(i))
    df_taxa['OTU']=otu_index 
    taxa_cols=[col for col in df_taxa.columns if 'OTU' not in col]
    for col in taxa_cols:
        df_taxa.at[df_taxa.index[-1], col] = 'Other'
    return df_taxa

def trim_taxa_names(x):
    '''Removes leading characters before taxa ID e.g. s__ '''
    match = re.sub(r'^[kpcofgs]__',"",str(x))
    return match

def add_otu_primary_key(df):
    '''Adds otu primary key column to dataframe'''
    df2 = df.copy()
    otu_index = []
    for i in range(0, len(df2)):
        otu_index.append("OTU"+str(i))
    df2['OTU']=otu_index
    df2.drop(columns=df.columns[0], axis=1,  inplace=True)
    return df2

In [12]:
metaphlan_data = '/home/sonec_iv/metaphlan.txt'
metaphlan_table =  pd.read_csv(metaphlan_data, sep='\t', header=0, skiprows=1)
metaphlan_table.head(20)

,clade_name,MPAR_124,MPAR_125,MPAR_126,MPAR_127,MPAR_128,MPAR_129,MPAR_132,MPAR_133,MPAR_200,...,MPAR_79,MPAR_80,MPAR_81,MPAR_83,MPAR_87,MPAR_88,MPAR_89,MPAR_90,MPAR_91,MPAR_93
0,k__Bacteria,100.00000,100.00000,99.98855,100.00000,100.00000,99.99233,98.14184,100.00000,99.88118,...,99.83424,100.00000,100.00000,100.00000,100.00000,99.98447,100.00000,100.00000,99.41350,100.00000
1,k__Bacteria|p__Bacteroidota,33.15072,25.21423,38.06827,37.86640,48.50238,36.99641,29.22813,35.46199,35.65467,...,49.49407,41.05625,30.88845,30.93893,2.83916,35.14729,21.80526,15.96523,40.59508,53.09683
2,k__Bacteria|p__Firmicutes,17.84881,7.85364,16.37042,16.88226,10.49326,14.96188,24.86213,21.13764,13.24656,...,12.13611,12.54588,12.94561,13.39367,4.32384,20.97348,7.61914,45.59132,10.48352,14.12618
3,k__Bacteria|p__Fusobacteria,13.81946,12.40443,6.44390,8.77256,13.07135,4.21855,1.42624,17.30598,12.50162,...,2.67649,5.24120,11.67282,4.02308,0.10361,10.04491,10.48314,6.56147,8.97633,8.10445
4,k__Bacteria|p__Actinobacteria,11.24452,24.42453,14.03526,9.59249,2.17765,1.39216,6.18004,4.33096,3.13341,...,1.33280,3.79651,17.47680,20.34730,89.73547,3.07394,37.06852,21.29466,1.83399,3.03869
5,k__Bacteria|p__Spirochaetes,10.51772,5.86466,17.34950,14.17450,13.44183,34.19308,8.65290,8.75861,11.26052,...,18.62271,21.65221,15.45101,1.64007,0.00000,12.45266,9.34012,6.90618,7.42401,15.64582
6,k__Bacteria|p__Candidatus_Saccharibacteria,6.62021,4.13092,2.33427,2.03188,1.94902,0.52717,8.80537,5.23078,0.65159,...,6.38269,1.37682,5.42436,0.78154,0.00000,5.68390,5.41738,0.30474,0.22677,0.36122
7,k__Bacteria|p__Proteobacteria,5.01000,16.22372,1.98669,5.79966,3.25875,3.65715,8.71633,3.57934,16.16897,...,5.12736,7.76072,3.42634,28.83855,2.99792,11.01112,6.78045,2.46745,28.31661,4.36169
8,k__Bacteria|p__Synergistetes,1.10055,0.26935,0.38688,1.56442,3.29806,1.21422,4.74422,0.77248,1.07620,...,0.65812,3.50460,0.24312,0.03685,0.00000,0.56336,0.09124,0.23719,0.21818,0.68113
9,k__Bacteria|p__Tenericutes,0.67232,1.10890,2.88410,2.75216,1.91089,2.55985,3.51873,3.06304,3.92798,...,2.50686,3.06582,2.10932,0.00000,0.00000,0.61373,1.22813,0.54687,0.34080,0.51614


In [9]:
metaphlan_table.columns

Index(['clade_name', 'MPAR_124', 'MPAR_125', 'MPAR_126', 'MPAR_127',
       'MPAR_128', 'MPAR_129', 'MPAR_132', 'MPAR_133', 'MPAR_200', 'MPAR_201',
       'MPAR_203', 'MPAR_205', 'MPAR_208', 'MPAR_210', 'MPAR_211', 'MPAR_212',
       'MPAR_214', 'MPAR_215', 'MPAR_77', 'MPAR_78', 'MPAR_79', 'MPAR_80',
       'MPAR_81', 'MPAR_83', 'MPAR_87', 'MPAR_88', 'MPAR_89', 'MPAR_90',
       'MPAR_91', 'MPAR_93'],
      dtype='object')

In [13]:
taxa_df = get_taxa_columns(metaphlan_table, 'Strain')
taxa_df.to_csv('metaphlan_taxa_df.csv', sep='\t', index=False)

In [14]:
count_df = add_otu_primary_key(metaphlan_table)
count_df.to_csv('metaphlan_count_df.csv', sep='\t',index=False)